In [4]:
# notes : the model should be robust for both male and female speakers , and all age groups



# To do 

# create dataset 
# use youtube-dl to download audio, use sox to convert audio to 16k mono format 
# download robust audio samples

# create tensorflow model

# create json dataset file , and dataloader


# ________________________________________________________

# ideas 

# use MFCC, total energy and F0






In [28]:
import sys

sys.path.insert(0, '/media/saurabh/New Volume/tf_audio_sentiment/pyAudioAnalysis')

from  pyAudioAnalysis  import audioBasicIO
from  pyAudioAnalysis import audioFeatureExtraction

In [12]:
import tensorflow as tf
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
from tensorflow.contrib import rnn
import scipy.io.wavfile as wav
import numpy as np
import os

directory_path='/media/saurabh/New Volume/tf_audio_sentiment/data/'
directory = os.fsencode(directory_path)

model_path = "/media/saurabh/New Volume/tf_audio_sentiment/models/rnn_model.ckpt"

In [13]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [14]:
# Parameters
learning_rate = 0.001
training_epochs = 25
batch_size = 1
display_step = 1

In [15]:
timesteps=2000 # max timesteps
num_input = 13 # size of vector, at each timestep
num_classes = 2 # happy or sad

num_hidden = 110 # hidden layer num of features

x = tf.placeholder(tf.float32, [None, timesteps, num_input]) 
y = tf.placeholder(tf.int32, [None]) #  Index of output , 0 = sad , 1 = happy

In [16]:
# define a function, that pads audio so that audio frames = max frames

def pad(input):
   # print(input.shape[0])
     if input.shape[0] < max_frames:
        
        diff = max_frames - input.shape[0]
        
        # pad and return input
        return np.pad(input,((0,diff),(0,0)), mode="constant")
    
     else:
        
        return input
        

In [17]:
# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of forward + backward cells
    'out': tf.Variable(tf.random_normal([2*num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}


In [18]:
def BiRNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, num_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
    x = tf.unstack(x, timesteps, 1)

    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    try:
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                              dtype=tf.float32)
    except Exception: # Old TensorFlow version only returns outputs not states
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                        dtype=tf.float32)
        
    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [19]:
Logits = BiRNN(x, weights, biases)
# prediction = tf.nn.softmax(logits) # use only for inference, softmax included in loss function


In [20]:
# Minimize error using cross entropy
#cost = tf.reduce_mean(-tf.reduce_sum(y_one_hot*tf.log(pred), reduction_indices=1))

cost = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=Logits)

# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [21]:
# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [22]:
# Start training
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        
        
         # Loop over all files
        for file in os.listdir(directory):
            filename = os.fsdecode(file)
            if filename.endswith(".wav"): 
                 print(filename)
                 accoustic_features=pad(audio_to_mfcc(directory_path + filename))
                 
                 
                    
                 print(accoustic_features.shape)
                
                 txt_file=filename.replace('.wav','.txt')
                 with open(directory_path + txt_file, 'r') as myfile:
                    data=myfile.read().replace('\n', '')
                    y_digit=int(data)
                    print(y_digit)
            
                    # Fit training using single example data
                    _, c = sess.run([optimizer, cost], feed_dict={x: accoustic_features,
                                                          y: [y_digit]})
            
                 
                    print(c)    
        
                 continue
            else:
                continue
        
        
     
            
           
        # Display logs per epoch step
      #  if (epoch+1) % display_step == 0:
       #     print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    
    print ("Optimization Finished!")
    
    # Save model weights to disk
    save_path = saver.save(sess, model_path)
    print("Model saved in file: %s" % save_path)
          

    # Test model
 #   correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
  #  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  #  print "Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]})

boy.wav


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-10c3877ca5bb>", line 15, in <module>
    accoustic_features=pad(audio_to_mfcc(directory_path + filename))
  File "<ipython-input-16-b6ccc63a812b>", line 5, in pad
    if input.shape[0] < max_frames:
NameError: name 'max_frames' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1821, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/IPython/core/ultratb.

NameError: name 'max_frames' is not defined